In [2]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as df
import yfinance as yf
import pandas_ta
import warnings
#warnings.filterwarnings('ignore')

In [35]:
# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# symbols_list = sp500['Symbol'].unique().tolist()

# end_date = '2024-11-25'

# start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

# df = yf.download(tickers=symbols_list, 
#                  start=start_date,
#                  end=end_date).stack()

#df.index.names = ['date', 'ticker']

#df.columns = df.columns.str.lower()

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-11-28 A        40.981747   43.599998   44.689999   43.580002   44.520000   
           AAPL     25.831560   27.892500   28.117500   27.847500   27.857500   
           ABBV     42.172958   59.459999   59.939999   59.009998   59.130001   
           ABT      33.519829   38.740002   39.220001   38.700001   39.220001   
           ACGL     25.870779   27.206667   27.406668   27.116667   27.343332   
...                       ...         ...         ...         ...         ...   
2024-11-22 XYL     126.870003  126.870003  127.730003  125.760002  126.139999   
           YUM     135.270004  135.270004  135.869995  134.179993  134.529999   
           ZBH     110.220001  110.220001  110.370003  108.669998  108.669998   
           ZBRA    397.190002  397.190002  397.839996  392.540009  394.279999   
           ZTS     176.960007  176.960007  178.070007  176.279999  176.350006   

Price                   volume  
date       ticker               
2016-11-28 A         2241800.0  
           AAPL    108776000.0  
           ABBV      6274600.0  
           ABT       5727800.0  
           ACGL      1845000.0  
...                        ...  
2024-11-22 XYL       1365600.0  
           YUM        992500.0  
           ZBH       2015000.0  
           ZBRA       232000.0  
           ZTS       1854600.0  

[989262 rows x 6 columns]

## Calculate features and technical indicators for each stock.
##### Garman-Klass Volatlity
##### RSI
##### Bollinger Bands
##### ATR
##### MACD 
##### Dollar Volume